# Natural Language Processing Course Project
### Politecnico di Milano, A.Y. 2024/2025

This notebook has been created for the Natural Language Processing course, held by Prof. Carman.

**Authors**:
* Matteo Figini
* Riccardo Figini
* Samuele Forner
* Caterina Motti
* Simone Zacchetti

The purpose of this notebook is to apply different NLP techniques to analyse the **RecipeNLG** dataset, you can find the dataset at the following [link](https://huggingface.co/datasets/mbien/recipe_nlg).

In [ ]:
# Import useful libraries
import numpy as np
import pandas as pd
import os
import ast
from collections import defaultdict
import matplotlib.pyplot as plt
import random

# Preliminary analysis

In this section, we aim to perform a preliminary analysis of the dataset.
- Load and inspect the dataset: analyze which field are present, missing field or inconsistencies. 
- Analyze recipe distribution: compute statistics and visualize them.
- Analyze entities distribution: compute statistics and visualize them.

### Load and inspect the dataset

In [ ]:
# Load the dataset from CSV file and show its head
df = pd.read_csv("/kaggle/input/recipenlg/dataset/full_dataset.csv")
df

In [ ]:
# Show the name of the columns
print("Columns:", df.columns.tolist())

Since the first column has no name, we renamed it:

In [ ]:
df = df.rename(columns={'Unnamed: 0': 'id'})

In [ ]:
# Dataset length
print(f"In the dataset there are {len(df)} different recipes!")

As we can see, the dataset contains 2231142 entries in 7 columns:
- id (int): ID.
- title (str): title of the recipe.
- ingredients (list of str): ingredients.
- directions (list of str): instruction steps.
- link (str): URL link.
- source (ClassLabel): origin of each recipe record, with possible value {"Gathered", "Recipes1M"}:
    - "Gathered" (0): additional recipes gathered from multiple cooking web pages using automated scripts in a web scraping process.
    - "Recipes1M" (1): recipes from the "Recipe1M+" dataset.
- NER (list of str): NER food entities.

Now, we check for **inconsistencies** and remove them if needed.

In [ ]:
# Rows with missing values
rows_with_null = df.isnull().any(axis=1).sum()
print(f"There are {rows_with_null} rows with at least one null value.")
print(df.isnull().sum())

The missing value is a title, which is not a critical field. However, we prefer to remove it.

In [ ]:
# Delete rows with at least one missing value
df.dropna()

In [ ]:
# Find duplicates by title
duplicate_counts = df['title'].value_counts()
print("Duplication results by: ", duplicate_counts)

Here we have lots of duplicates. However, they might represent different variations of the same recipe which could be useful for further analysis.

Since the dataset is really big (2M+ entries), we decided to make a random subsampling of it by retaining only 25% (around half million entries).

In [ ]:
# Randomly sample the dataset
df = df.sample(frac=0.25, random_state=42)

### Analyze vocaboulary

In [ ]:
# Import necessary libraries
import re
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk import bigrams

# Download necessary NLTK data
nltk.download('stopwords')

In [ ]:
# Define a new column 'text' with directions as a string
df['text'] = df['directions'].apply(lambda x: ' '.join(ast.literal_eval(x)))
all_instructions_list = df['text'].tolist()

# Preprocess the text and tokenize it into words
full_text = ' '.join(all_instructions_list)
full_text = full_text.lower()
full_text = re.sub(r'[^a-zA-Z0-9\s]', '', full_text)
words = full_text.split()

# Remove stopwords and tokenize
stop_words = set(stopwords.words('english'))
words_nostopwords = [w for w in words if w not in stop_words]
tokens = [w for w in words_nostopwords]

# Generate bigrams and frequency distribution
bigram_list = list(bigrams(tokens))
bigram_fdist = FreqDist(bigram_list)

# Frequency distribution of words
fdist = FreqDist(words_nostopwords)

In [ ]:
# Vocabulary size
vocabulary_size = len(fdist)
print("Vocabulary size: ", vocabulary_size)

# Average number of unique words per recipe
unique_words_per_recipe = []

for directions in df['text']:
    directions = directions.lower()
    directions = re.sub(r'[^a-zA-Z0-9\s]', '', directions)  # remove punctuation
    tokens = directions.split()  # split into words
    tokens = [w for w in tokens if w not in stop_words]
    unique_words = set(tokens)
    unique_words_per_recipe.append(len(unique_words))

avg_unique_words_per_recipe = sum(unique_words_per_recipe) / len(unique_words_per_recipe)    
print("Average number of unique words per recipe: ", avg_unique_words_per_recipe)

# Compute word length distribution
word_lengths = [len(word) for word in words_nostopwords]
avg_word_length = sum(word_lengths) / len(word_lengths)
min_word_length = min(word_lengths)
max_word_length = max(word_lengths)

print("Average word length: ", avg_word_length, " characters")
print("Minimum word length: ", min_word_length, " characters")
print("Maximum word length: ", max_word_length, " characters")

The high vocabulary size shows that the dataset is very rich and diverse, covering many ingredients, actions, and contextual descriptions. 

However the average number of unique words in each recipe is quite small: this suggests that each recipe tends to use a fairly concise and domain-specific subset.

NB: the maximum word length (262 characters) is likely an outlier. We must investigate it further. 

In [ ]:
# Find the longest words
def find_long_words_in_directions(df):
    long_words = []
    
    for directions in df['text']:
        # Extract words longer than 45 characters, since in English vocabulary the longest word has 45 characters
        long_words.extend([word for word in directions.split() if len(word) > 45])
    
    if long_words:
        print("Found ", len(long_words), " long words in the dataset (greater than 45 characters).")
        print("Printing the first 10 for brevity: \n")
        print(long_words[:10])
    else:
        print("No words longer than 45 characters found.")

find_long_words_in_directions(df)

Most of them are links, so we can keep them (also considering that they are not that frequent).

In [ ]:
# Create frequency distribution for bigrams
bigram_fdist = FreqDist(bigram_list)

# Display top 10 most common words
print("\nTop 10 most common words:")
for word, freq in fdist.most_common(10):
    print(f"{word}: {freq}")

# Display top 10 most common bigrams
print("\nTop 10 most common bigrams:")
for bigram, freq in bigram_fdist.most_common(10):
    print(f"{' '.join(bigram)}: {freq}")


### Analyze recipes distribution
Since the recipes are the core of the dataset, it may be useful to analyse the length of each recipe to plot their distribution.

Recipes with very high or very low word counts might be outliers. Analyzing the distribution allows us to detect these outliers, which might skew our analysis. For instance, a recipe with only one word might be a mistake.

In [ ]:
# Define a new column to count the number of words in 'directions'
df['dir_word_count'] = df['directions'].apply(lambda x: len(' '.join(x).split()))

In [ ]:
# Retrieve the length and index of the longest and shortest recipe
max_words = df['dir_word_count'].max()
min_words = df['dir_word_count'].min()

max_words_idx = df['dir_word_count'].idxmax()
min_words_idx = df['dir_word_count'].idxmin()
print(f"Recipes' direction have between {min_words} and {max_words} words.")
print(f"The longest recipe has {max_words} words and it stands at location {max_words_idx}.")
#print(df['directions'].loc[max_words_idx])
print(df[['title','directions', 'dir_word_count']].loc[min_words_idx])

In [ ]:
# Visualize the distribution of the number of words in column "directions" to show how long recipes are (in number of words)
plt.figure(figsize=(12,6))
plt.hist(df['dir_word_count'], bins=max_words, range=(0, max_words))
plt.title('Distribution of words in column \"directions\"')
plt.xlabel('Number of words in the recipe')
plt.ylabel('Number of recipes')
plt.grid(True)
plt.show()

In [ ]:
# Analyzing recipe word count distribution
word_counts = df['dir_word_count'].dropna()
mean_word_count = word_counts.mean()
median_word_count = word_counts.median()
percentile_90 = word_counts.quantile(0.9)
percentile_95 = word_counts.quantile(0.95)

print(f"Mean word count: {mean_word_count}")
print(f"Median word count: {median_word_count}")
print(f"90th percentile: {percentile_90}")
print(f"95th percentile: {percentile_95}")

In [ ]:
# Count recipes with less than upper_threshold words
upper_threshold = 300
count_less_threshold = (word_counts <= upper_threshold).sum()

upper_percentage = count_less_threshold/len(df) * 100

print(f"{round(upper_percentage, 2)}% of the recipes have at most {upper_threshold} words.")

In [ ]:
# Count recipes with more than lower_threshold words
lower_threshold = 20
count_more_threshold = (word_counts >= lower_threshold).sum()

lower_percentage = count_more_threshold/len(df) * 100

print(f"{round(lower_percentage, 2)}% of the recipes have at least {lower_threshold} words.")

### Analyze entities distribution
The aim of this section is to analyse the number of elements in the NER column, which contains the number of entities in each recipe, and to provide a distribution, giving a rough measure of the "complexity" of the recipe.

In [ ]:
# Define a new column containing the number of entities of each recipe
ner_column = 'ner_len'
df[ner_column] = df['NER'].apply(lambda x: len(ast.literal_eval(x)))

In [ ]:
# Retrieve the length and index of recipe with highest and lowest NER number
max_ner_len = df[ner_column].max()
min_ner_len = df[ner_column].min()

max_ner_len_idx = df[ner_column].idxmax()
print(f"Recipes have between {min_ner_len} and {max_ner_len} entities.")

print(f"The 'biggest' recipe has {max_ner_len} entities and it is at location {max_ner_len_idx}.")
#print(df['NER'].loc[max_ner_len_idx])

In [ ]:
# Visualize the number of entities in each recipe
plt.figure(figsize=(12,6))
plt.hist(df[ner_column], bins=max_ner_len, range=(0, max_ner_len))
plt.title('Distribution of the number of entities')
plt.xlabel('Number of entities')
plt.ylabel('Number of recipes')
plt.grid(True)
plt.show()

In [ ]:
# Analyzing recipe NER count distribution
entities_counts = df[ner_column].dropna()
mean_entities_count = entities_counts.mean()
median_entities_count = entities_counts.median()
percentile_90 = entities_counts.quantile(0.9)
percentile_95 = entities_counts.quantile(0.95)

print(f"Mean NER count: {mean_entities_count}")
print(f"Median NER count: {median_entities_count}")
print(f"90th percentile: {percentile_90}")
print(f"95th percentile: {percentile_95}")

In [ ]:
# Count recipes with less than upper_threshold entities
upper_threshold = 25
count_less_threshold = (entities_counts <= upper_threshold).sum()

upper_percentage = count_less_threshold/len(df) * 100

print(f"{round(upper_percentage, 2)}% of the recipes have at most {upper_threshold} entities.")

In [ ]:
# Count recipes with more than lower_threshold entities
lower_threshold = 3
count_more_threshold = (entities_counts >= lower_threshold).sum()

lower_percentage = count_more_threshold/len(df) * 100

print(f"{round(lower_percentage, 2)}% of the recipes have at least {lower_threshold} entities.")

In [ ]:
# Filter recipes where there are NO NER
zero_NER_recipes = df[df['ner_len'] == 0]

print(zero_NER_recipes[['title', 'NER']])
# Drop recipes where there are no NERs
df = df[df['ner_len'] != 0]

It means that the NER process didn't detect any entities for these recipes. This could happen for various reasons, such as:
- The recipe doesn't contain recognizable entities like ingredients, amounts, or special terms.
- The NER process might not be comprehensive enough to detect all relevant entities in the recipe.
- The recipe could be incomplete or malformed in a way that prevents proper entity recognition.

For this reason, we decided to drop these rows.

NB: It may happens that there are NO entries without NER, since we are using a subsample of the original dataset. 

## Clustering

Now we want to perform some clustering analysis.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
titles = df['title'].astype(str)
X = vectorizer.fit_transform(titles)

In [ ]:
inertia = []
k_values = list(range(2, 50))

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init='auto')
    kmeans.fit(X)
    inertia.append(kmeans.inertia_)

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(k_values, inertia, marker='o')
plt.title('Elbow method to determine k')
plt.xlabel('Number of clusters (k)')
plt.ylabel('Intertia')
plt.xticks(k_values)
plt.grid(True)
plt.show()

It seems that two interesting values for K are 13 and 21, let's try both of them.

In [ ]:
n_clusters = 13
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
kmeans.fit(X)

# Add clusters to the original dataframe
df['cluster'] = kmeans.labels_

# Print an example of titles grouped by cluster
for i in range(n_clusters):
    print(f"\nCluster {i}:")
    print(df[df['cluster'] == i]['title'].head(5).to_string(index=False))

In [ ]:
# la eliminiamo?
# Get the centroid for the first cluster
centroid = kmeans.cluster_centers_[0]

# Sort terms according to their weights
# (argsort goes from lowest to highest, we reverse the order through slicing)
sorted_terms = centroid.argsort()[::-1]

# Print out the top 10 terms for the cluster
[titles.iloc[j] for j in sorted_terms[:10]]

In [ ]:
print("Top terms per cluster:")
vocab = vectorizer.get_feature_names_out()

for i in range(kmeans.n_clusters):
    centroid = kmeans.cluster_centers_[i]
    sorted_terms = centroid.argsort()[::-1]
    print(f"Cluster {i}:\t{[vocab[j] for j in sorted_terms[:5]]}")

In [ ]:
print('Number of docs in: ')

for i in range(kmeans.n_clusters):
    print(f"Cluster {i}: {np.sum(kmeans.labels_ == i)}")

In [ ]:
n_clusters = 21
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init='auto')
kmeans.fit(X)

df['cluster'] = kmeans.labels_

for i in range(n_clusters):
    print(f"\nCluster {i}:")
    print(df[df['cluster'] == i]['title'].head(5).to_string(index=False))

In [ ]:
# la rimuoviamo?
centroid = kmeans.cluster_centers_[0]

sorted_terms = centroid.argsort()[::-1]

[titles.iloc[j] for j in sorted_terms[:20]]

In [ ]:
print("Top terms per cluster:")
vocab = vectorizer.get_feature_names_out()

for i in range(kmeans.n_clusters):
    centroid = kmeans.cluster_centers_[i]
    sorted_terms = centroid.argsort()[::-1]
    print(f"Cluster {i}:\t{[vocab[j] for j in sorted_terms[:5]]}")

In [ ]:
print('Number of docs in: ')

for i in range(kmeans.n_clusters):
    print(f"Cluster {i}: {np.sum(kmeans.labels_ == i)}")

### Evaluation

In order to evaluate our clustering we are using only intrinsic method, since we do not have the real labels of the clusters.

In [ ]:
from sklearn import metrics

print("Intrinsic evaluation measures:")
print("Within-cluster sum-of-squares:", str(kmeans.inertia_))
#print("Silhouette coefficient:", str(metrics.silhouette_score(X, kmeans.labels_)))

Since the dataset is quite big, even reducing it to a quarter of the original, we try to perform also a Mini Batch Clustering.

In [ ]:
from sklearn.cluster import MiniBatchKMeans

n_clusters = 21
mb_kmeans = MiniBatchKMeans(n_clusters=n_clusters,batch_size=500, random_state=2307)
mb_kmeans.fit(X)

In [ ]:
print("Intrinsic evaluation measures:")
print("Within-cluster sum-of-squares:", str(mb_kmeans.inertia_))
#print("Silhouette coefficient:", str(metrics.silhouette_score(X, mb_kmeans.labels_)))

Now, let's try to visualize clusters.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(3)
reduced_data = svd.fit_transform(X)

[x,y,z] = np.transpose(reduced_data)
[x,y,z]

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c=kmeans.labels_, marker='.')

We have also tried to perform clustering for the NER column, performing the 'elbow method' and plotting the results. However, the plot has not highlighter any points in which improvement in the measure decreases substantially from one time step to the next. 

## Indexing
We decided to index the recipes based on the directions, in order to perform keyword search over them. We choose the directions since they will likely contains both ingredients and keywords relative to the procedure (e.g. baked, fried).

In [ ]:
!pip install -q python-terrier==0.11.0

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pyterrier as pt

if not pt.started():
  pt.init()

In [ ]:
# Create a list of documents (use 'title' for indexing)
documents_title = [{'docno': str(i), 'text': text} for i, text in enumerate(df['title'])]

# Create the index
indexer = pt.IterDictIndexer("./index_title")
indexer.index(documents_title)

In [ ]:
# Create a list of documents (use 'directions' for indexing)
documents_directions = [{'docno': str(i), 'text': text} for i, text in enumerate(df['directions'])]

# Create the index
indexer = pt.IterDictIndexer("./index_directions")
indexer.index(documents_directions)

In [ ]:
# Create documents with multiple fields
documents_fields = [
    {
        'docno': str(i),
        'title': row['title'],
        'ingredients': row['ingredients'],
        'directions': row['directions']
    }
    for i, row in df.iterrows()
]

# Index the fielded documents
indexer_fields = pt.IterDictIndexer("./index_fields")

# Set meta fields and indexed fields
indexref = indexer_fields.index(
    documents_fields,
    fields=["title", "ingredients", "directions"],  
    meta={'docno': 20, 'title': 512, 'ingredients': 1024, 'directions': 4096}
)

In [ ]:
# Create a searcher using the index
index_title = pt.IndexFactory.of("./index_title")
index_directions = pt.IndexFactory.of("./index_directions")
index_fields = pt.IndexFactory.of(indexref)

# BM25 retrieval model
bm25_tit = pt.terrier.Retriever(index_title, wmodel="BM25")
bm25_dir = pt.terrier.Retriever(index_directions, wmodel="BM25")

# TF-IDF retrieval model
tfidf_tit = pt.terrier.Retriever(index_title, wmodel="TF_IDF")
tfidf_dir = pt.terrier.Retriever(index_directions, wmodel="TF_IDF")

# DFRee retrieval model (Document Frequency based)
dfree_tit = pt.terrier.Retriever(index_title, wmodel="DFRee")
dfree_dir = pt.terrier.Retriever(index_directions, wmodel="DFRee")

# Create BM25F retriever with field weights
# Weighted BM25 over each field
bm25_title = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "title"], field="title")
bm25_ingredients = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "ingredients"], field="ingredients")
bm25_directions = pt.BatchRetrieve(index_fields, wmodel="BM25", controls={"w": "1.0"}, metadata=["docno", "directions"], field="directions")

# Weighted combination of scores: BM25F-like
bm25f_manual = (
    bm25_title * 0.3 +
    bm25_ingredients * 0.4 +
    bm25_directions * 0.3
)

By using all of these models, we can perform a comprehensive evaluation of the different retrieval strategies. Each model captures different aspects of relevance:

- BM25 accounts for both term and document frequencies, providing a balanced relevance score.
- TF-IDF emphasizes the importance of rare terms across the document collection.
- DFRee is useful for understanding how document frequency affects relevance.

We also tried BM25 without term frequency (no tf), but it performed to same as classic BM25. This is likely because in our dataset, document length and term frequency did not vary significantly across relevant documents, so removing the tf component had minimal impact on ranking.
To keep the notebook clean we omitted this variant.

In [ ]:
print(index_title.getCollectionStatistics().toString())
print(index_directions.getCollectionStatistics().toString())
print(index_fields.getCollectionStatistics().toString())

In [ ]:
# Prepare some queries and evaluate them
queries = [
    {"qid": "1", "query": "chicken casserole"},
    {"qid": "2", "query": "simple birthday cake"},
    {"qid": "3", "query": "baked salmon"},
]

titles = df['title']
plot_data = []

# Loop through each query
for q in queries:
    print(f"\n=== Query {q['qid']}: {q['query']} ===")

    # Run all models for the titles including BM25F
    result_bm25_tit = bm25_tit.search(q['query'])
    result_tfidf_tit = tfidf_tit.search(q['query'])
    result_dfree_tit = dfree_tit.search(q['query'])   

    # Add all model results to the loop
    for method_name, result in [
        ("BM25", result_bm25_tit),
        ("TF-IDF", result_tfidf_tit),
        ("DFRee", result_dfree_tit),
    ]:
        print(f"\n--- Method: {method_name} ---")

        for rank, (docno, score) in enumerate(zip(result["docno"][:5], result["score"][:5])):
            docno_int = int(docno)  # Convert from str to int
            title = df['title'].iloc[docno_int] if docno_int < len(df) else "TITLE NOT FOUND"
            print(f"DocNO: {docno:<7} | Title: {title:<50.48} | Score: {score:.4f}")
            
            # Append to plotting data
            plot_data.append({
                'Query': q['query'],
                'Query_ID': q['qid'],
                'Model': method_name,
                'Rank': rank + 1,
                'Docno': docno_int,
                'Title': title,
                'Score': score
            })

In [ ]:
# Visualize the results of the third query
plot_df = pd.DataFrame(plot_data)
df_first = plot_df[plot_df['Query_ID'] == "3"]

plt.figure(figsize=(10, 6))

for model in df_first['Model'].unique():
    md = df_first[df_first['Model'] == model]
    plt.plot(md['Rank'], md['Title'], marker='o', linestyle='-', label=model)

plt.xlabel('Rank')
plt.ylabel('Recipe title')
plt.title(f'Results for Query "{queries[2]["query"]}"')
plt.legend(title='Model')
plt.xticks([1, 2, 3, 4, 5])
plt.grid(True)
plt.show()

In this case, BM25 and TF-IDF yielded the same document rankings: this happen when documents are short, uniformly sized, or when term frequencies are relatively balanced, making the additional complexity of BM25 less impactful.

This suggest that relying only on document titles for indexing provides limited term variability and context, reducing the effectiveness of retrieval models. Incorporating richer textual content would likely yield more discriminative and informative rankings.

In [ ]:
# Prepare some queries and evaluate them
queries = [
    {"qid": "1", "query": "chicken casserole"},
    {"qid": "2", "query": "simple birthday cake"},
    {"qid": "3", "query": "baked salmon"},
]

titles = df['title']
plot_data = []

# Loop through each query
for q in queries:
    print(f"\n=== Query {q['qid']}: {q['query']} ===")

    # Run all models for the directions including BM25F
    result_bm25_dir = bm25_dir.search(q['query'])
    result_tfidf_dir = tfidf_dir.search(q['query'])
    result_dfree_dir = dfree_dir.search(q['query'])
    
    result_bm25f = bm25f_manual.search(q['query'])  

    # Add all model results to the loop
    for method_name, result in [
        ("BM25", result_bm25_dir),
        ("TF-IDF", result_tfidf_dir),
        ("DFRee", result_dfree_dir),
        ("BM25F", result_bm25f) 
    ]:
        print(f"\n--- Method: {method_name} ---")

        for rank, (docno, score) in enumerate(zip(result["docno"][:5], result["score"][:5])):
            docno_int = int(docno)  # Convert from str to int
            title = df['title'].iloc[docno_int] if docno_int < len(df) else "TITLE NOT FOUND"
            print(f"DocNO: {docno:<7} | Title: {title:<50.48} | Score: {score:.4f}")
            
            # Append to plotting data
            plot_data.append({
                'Query': q['query'],
                'Query_ID': q['qid'],
                'Model': method_name,
                'Rank': rank + 1,
                'Docno': docno_int,
                'Title': title,
                'Score': score
            })

In [ ]:
# Visualize the results of the third query
plot_df = pd.DataFrame(plot_data)
df_first = plot_df[plot_df['Query_ID'] == "3"]

plt.figure(figsize=(10, 6))

for model in df_first['Model'].unique():
    md = df_first[df_first['Model'] == model]
    plt.plot(md['Rank'], md['Title'], marker='o', linestyle='-', label=model)

plt.xlabel('Rank')
plt.ylabel('Recipe title')
plt.title(f'Results for Query "{queries[2]["query"]}"')
plt.legend(title='Model')
plt.xticks([1, 2, 3, 4, 5])
plt.grid(True)
plt.show()

About the results:

- TF-IDF put “Balsamic Glazed Salmon” at #1: we can see that TF-IDF’s emphasis on rare terms may be boosting “balsamic”.
- DFRee puts “Fennel-And-Dill-Rubbed Grilled Salmon” at #1, and then “Salmon Wrapped In Fig Leaves With Baked Kale” at #2: it shows that rarer, longer titles win more weight under DFRee.
- Unlike single-field models, BM25F combines weighted evidence from different parts of the document, making it robust when key terms appear in both title, ingredients and directions.

## Word2Vec

Training a Word2Vec embedding on the RecipeNLG dataset allows us to capture the semantic relationships between words in cooking instructions. Word2Vec learns to represent words as dense vectors, where words with similar meanings are positioned closer together in the vector space. This enables us to explore word similarities, identify common patterns, and improve various natural language processing tasks like recipe recommendation and ingredient analysis.

In [ ]:
# Convert the 'text' column of the DataFrame into a list of strings (e.g., recipe directions)
w2v_directions = list(df['text'])

# Tokenize each direction 
tokenized_directions = [re.sub('\W', ' ', direction).lower().split() for direction in w2v_directions]

Let's remove the stopwords
(we need a context window to train it, so we are discarding the directions composed by only one word)

In [ ]:
filtered_directions = [[word for word in sublist if word not in stop_words] for sublist in tokenized_directions]
filtered_directions = [direction for direction in filtered_directions if len(direction) > 1]
len(filtered_directions)

Now it's time to  train our model!


In [ ]:
from gensim.models.word2vec import Word2Vec

model = Word2Vec(filtered_directions, vector_size=50, min_count=5, window=10)

len(model.wv)

Ever wondered what is a possible vector representation of the word soup?  Me neither!

In [ ]:
term = 'soup'
model.wv[term]

Or what are the most similar words to 'chicken' (Between the ones in our dataset)

In [ ]:
term = 'chicken'
model.wv.most_similar(term)

From 0 to 1, how  similar the words chicken and beef are?

In [ ]:
model.wv.similarity('chicken', 'beef')

chicken:breast = beef:?

In [ ]:
vec = model.wv['beef'] + model.wv['breast'] - model.wv['chicken']
model.wv.similar_by_vector(vec)

Right now we are working with embedding vectors of 50 dimensions. Unfortunately, the human brain is limited and can visualize only up to three of them. So it's necessary to reduce our vectors' dimensions to this number. Let's try to do so using t-distributed stochastic neighbor embedding (t-SNE):

In [ ]:
from sklearn.manifold import TSNE

samples = random.sample(list(model.wv.key_to_index), 1000)
vectors = model.wv[samples]

tsne = TSNE(n_components=3, n_iter=2000)
tsne_embedding = tsne.fit_transform(vectors)

x, y, z = np.transpose(tsne_embedding)

In [ ]:
import plotly.express as px

fig = px.scatter_3d(x=x[:300],y=y[:300],z=z[:300],text=samples[:300])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

Quite interesting isn't it? 

Numbers are near each other (probably they represent different oven temperatures). Also values like 9x5x3 and 2x3 (lengths of the baking trays?).

Same for fruits and vegetables

Why don't we try principal component analysis to perform the dimensionality reduction?

(and also sample again, you never know)

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
samples = random.sample(list(model.wv.key_to_index), 1000)
vectors = model.wv[samples]

pca = PCA(n_components=3)
pca_embedding = pca.fit_transform(vectors)

explained_variance_ratio = pca.explained_variance_ratio_
print(f"Explained variance ratio: {explained_variance_ratio}" )
print(f"Total explained variance: {sum(explained_variance_ratio)}")

In [ ]:
x, y, z = np.transpose(pca_embedding)

fig = px.scatter_3d(x=x[:200],y=y[:200],z=z[:200],text=samples[:200])
fig.update_traces(marker=dict(size=3,line=dict(width=2)),textfont_size=10)
fig.show()

The 3 dimensions that hold the most informations tent to agglomerate around the values (0,0,0), making pca reductions not suited for visualisation purposes


# Training models

In this section we are going to train some models, from the easiest to the most complex ones.

Firstly, we create a secreto token to use HuggingFace models.

In [ ]:
⁠from kaggle_secrets import UserSecretsClient

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("NLP project") ⁠


# Further development 

In [ ]:
We would like to see how some of our riceipts can look like...

In [ ]:
# Extract 3 titles to turn into images
sampled_titles = df['title'].sample(n=3, random_state=42).tolist()

In [ ]:
import torch
from huggingface_hub import login

login(secret_value_0)

In [ ]:
from diffusers import DiffusionPipeline

pipe = DiffusionPipeline.from_pretrained("stabilityai/stable-diffusion-xl-base-1.0", torch_dtype=torch.float16, use_safetensors=True, variant="fp16")
pipe.to("cuda")

# if using torch < 2.0
# pipe.enable_xformers_memory_efficient_attention()

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for i, title in enumerate(sampled_titles):
    image = pipe(prompt=title).images[0]
    axes[i].imshow(image)
    axes[i].set_title(title, fontsize=10)
    axes[i].axis('off')

In [ ]:
plt.tight_layout()
plt.show()